In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats
from google.colab import files
uploaded = files.upload()
import io

In [ ]:
Data = pd.read_csv(io.BytesIO(uploaded['diabetes.csv']))

In [ ]:
Data.head()

In [ ]:
Data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = Data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [ ]:
print(Data.isnull().sum())

In [ ]:
Data['Glucose'].fillna(Data['Glucose'].mean(), inplace = True)
Data['BloodPressure'].fillna(Data['BloodPressure'].mean(), inplace = True)
Data['SkinThickness'].fillna(Data['SkinThickness'].median(), inplace = True)
Data['Insulin'].fillna(Data['Insulin'].median(), inplace = True)
Data['BMI'].fillna(Data['BMI'].median(), inplace = True)

In [ ]:
print(Data.isnull().sum())

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X =  pd.DataFrame(sc.fit_transform(Data.drop(["Outcome"],axis = 1),),
        columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'])

In [ ]:
X.head()

In [ ]:
Y = Data['Outcome']

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 0)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

In [ ]:

import operator

#Euclidean Distance
def E_Distance(x1, x2):
    distance = np.square(x1 - x2)
    return np.sqrt(distance.sum(axis=1))

# K-NN model
def knn(trainingSet_x, testInstance,trainingset_y, k):
    dist = E_Distance(trainingSet_x,testInstance)
    distances={i:dist.iloc[i] for i in range(len(dist))}
    sortdist = sorted(distances.items(), key=operator.itemgetter(1))
    row_index=[sortdist[i][0] for i in range(k)]
    y_temp=stats.mode(trainingset_y.iloc[row_index].values)
    return(y_temp[0][0])

# supplying test data to the model
k=11
y_pred=[]
for i in range(len(X_test)):
    y_pred.append(knn(X_train, X_test.iloc[i],Y_train, k))
print(len(y_pred))

In [ ]:
#Confusion Matrix
tp=len([i for i in range(Y_test.shape[0]) if Y_test.iloc[i]==1 and y_pred[i]==1])
tn=len([i for i in range(Y_test.shape[0]) if Y_test.iloc[i]==0 and y_pred[i]==0])
fp=len([i for i in range(Y_test.shape[0]) if Y_test.iloc[i]==0 and y_pred[i]==1])
fn=len([i for i in range(Y_test.shape[0]) if Y_test.iloc[i]==1 and y_pred[i]==0])
confusion_matrix=np.array([[tp,fn],[fp,tn]])
print(confusion_matrix,type(confusion_matrix))

In [ ]:
tp=confusion_matrix[0,0]
fp=confusion_matrix[1,0]
tn=confusion_matrix[1,1]
fn=confusion_matrix[0,1]

Accuracy=(tp+tn)/(confusion_matrix.sum())*100

Precision=tp/(tp+fp)*100


print('Accuracy= %.2f'%Accuracy,'\nPrecision= %.2f'%Precision)